In [ ]:
!pip install text-hammer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 4.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
df = pd.read_csv('/content/multidomain.csv')
df

,Review,Sentiment
0,Looks even better in person. Be careful to not...,1
1,When you don't want to spend a whole lot of ca...,1
2,"so the case came on time, i love the design. I...",1
3,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,0
4,"I liked it because it was cute, but the studs ...",1
...,...,...
99995,Didn't come with a box and was pretty pricy fo...,0
99996,Not the same quality as was advertised. It ar...,0
99997,The order was a fast delivery. But at first co...,0
99998,Wtf I did not buy this just wondering how you ...,0


In [ ]:
import text_hammer as th

In [ ]:
%%time
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

CPU times: user 467 µs, sys: 0 ns, total: 467 µs
Wall time: 553 µs


In [ ]:
def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x: str(x).lower())
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.make_base(x))
    return(df)

In [ ]:
df_cleaned = text_preprocessing(df, 'Review')

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "http://www.amazon.com/gp/product/0972683275?redirect=true&ref_=cm_cr_ryp_prd_ttl_sol_5" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://www.amazon.com/dp/b000s56380/ref=cm_cr_ryp_prd_ttl_sol_3" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "http://www.ebay.com/itm/sega-genesis-3-system-console-controller-hookups-bundled-/381287963634?point=lh_defaultdomain_0&hash=item58c

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
df_cleaned

,Review,Sentiment
0,look even better in person be careful to not d...,1
1,when you do not want to spend a whole lot of c...,1
2,so the case come on time i love the design i a...,1
3,do not care for it give it as a gift and they ...,0
4,i like it because it was cute but the stud fal...,1
...,...,...
99995,do not come with a box and was pretty pricy fo...,0
99996,not the same quality as was advertise it arriv...,0
99997,the order was a fast delivery but at first cou...,0
99998,what the fuck i do not buy this just wonder ho...,0


In [ ]:
reviews = df_cleaned.Review
sentiment = df_cleaned.Sentiment

In [ ]:
newdf = df_cleaned

In [ ]:
import gensim
from gensim.utils import simple_preprocess
newdf['tokenized']=newdf['Review'].apply(simple_preprocess,deacc=True)
newdf.head()

,Review,Sentiment,tokenized
0,look even better in person be careful to not d...,1,"[look, even, better, in, person, be, careful, ..."
1,when you do not want to spend a whole lot of c...,1,"[when, you, do, not, want, to, spend, whole, l..."
2,so the case come on time i love the design i a...,1,"[so, the, case, come, on, time, love, the, des..."
3,do not care for it give it as a gift and they ...,0,"[do, not, care, for, it, give, it, as, gift, a..."
4,i like it because it was cute but the stud fal...,1,"[like, it, because, it, was, cute, but, the, s..."


In [ ]:
from gensim.models import Word2Vec
dim = 100
window = 3
min_count = 1
workers = 4 #threads
sg = 1 # skip gram model = 1
tokens = pd.Series(newdf['tokenized']).values
print(tokens)

[list(['look', 'even', 'better', 'in', 'person', 'be', 'careful', 'to', 'not', 'drop', 'your', 'phone', 'so', 'often', 'because', 'the', 'rhinestone', 'will', 'fall', 'off', 'duh', 'more', 'of', 'decorative', 'case', 'than', 'it', 'is', 'protective', 'but', 'will', 'say', 'that', 'it', 'fit', 'perfectly', 'and', 'securely', 'on', 'my', 'phone', 'overall', 'very', 'pleased', 'with', 'this', 'purchase'])
 list(['when', 'you', 'do', 'not', 'want', 'to', 'spend', 'whole', 'lot', 'of', 'cash', 'but', 'want', 'great', 'dealthis', 'is', 'the', 'shop', 'to', 'buy', 'from'])
 list(['so', 'the', 'case', 'come', 'on', 'time', 'love', 'the', 'design', 'am', 'actually', 'miss', 'stud', 'but', 'nothing', 'too', 'noticeable', 'the', 'studding', 'is', 'almost', 'bit', 'sloppy', 'around', 'the', 'bow', 'but', 'once', 'again', 'not', 'too', 'noticeable', 'have', 'not', 'put', 'in', 'my', 'phone', 'yet', 'so', 'this', 'is', 'just', 'what', 'have', 'notice', 'so', 'far'])
 ...
 list(['the', 'order', 'was'

In [ ]:
model = Word2Vec(tokens, min_count = min_count, size = dim, workers = workers, window = window, sg = sg)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(newdf['Review'],newdf['Sentiment'],test_size = 0.2,random_state=42)
X_train,X_val,Y_train,Y_val = train_test_split(X_train,Y_train,test_size = 0.0125,random_state=42)

In [ ]:
X_train

27156    i have been loan this book by a friend and imm...
76081                        picture quality was very poor
26129    good quality feature for the price would recom...
10074    gt the product for free basically purchase the...
75350    pay heed to mister carrey warn right at the ve...
                               ...                        
32231    buy mine at a barne and noble and it do t even...
96487    man roller coaster tycoon is awesome but this ...
20095    i suppose there is some dramatic action to thi...
30376    a interesting and enlighte book it put the pro...
50554    dietrich return to germany to tour in 1963 the...
Name: Review, Length: 79000, dtype: object

In [ ]:
df_train = pd.DataFrame({"review":X_train,"sentiment":Y_train})
df_val = pd.DataFrame({"review":X_val,"sentiment":Y_val})
df_test = pd.DataFrame({"review":X_test,"sentiment":Y_test})
df_train

,review,sentiment
27156,i have been loan this book by a friend and imm...,1
76081,picture quality was very poor,0
26129,good quality feature for the price would recom...,1
10074,gt the product for free basically purchase the...,0
75350,pay heed to mister carrey warn right at the ve...,0
...,...,...
32231,buy mine at a barne and noble and it do t even...,0
96487,man roller coaster tycoon is awesome but this ...,0
20095,i suppose there is some dramatic action to thi...,0
30376,a interesting and enlighte book it put the pro...,1


In [ ]:
from keras.preprocessing.text import Tokenizer

num_words = 10000 
tokenizer = Tokenizer(num_words, lower = True)
df_total = pd.concat([df_train['review'], df_test.review], axis = 0)
tokenizer.fit_on_texts(df_total)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train=tokenizer.texts_to_sequences(df_train['review']) # this converts texts into some numeric sequences 
X_train_pad = pad_sequences(X_train, maxlen = 300, padding='post') # this makes the length of all numeric sequences equal 
X_test = tokenizer.texts_to_sequences(df_test.review)
X_test_pad = pad_sequences(X_test, maxlen = 300, padding = 'post')
X_val = tokenizer.texts_to_sequences(df_val.review)
X_val_pad = pad_sequences(X_val, maxlen = 300, padding = 'post')
X_train_pad

array([[   2,   11,  105, ...,    0,    0,    0],
       [ 306,  141,   17, ...,    0,    0,    0],
       [  29,  141,  265, ...,    0,    0,    0],
       ...,
       [   2,  462,   53, ...,    0,    0,    0],
       [   5,  448,    3, ...,    0,    0,    0],
       [ 217,    4, 4968, ...,    0,    0,    0]], dtype=int32)

In [ ]:
import numpy as np

vector_size = 100
w2v_weight_matrix = np.zeros((num_words ,vector_size))
w2v_weight_matrix.shape

(10000, 100)

In [ ]:
for word, index in tokenizer.word_index.items():
    if index < num_words: # since index starts with zero 
        if word in model.wv.vocab:
            w2v_weight_matrix[index] = model[word]
        else:
            w2v_weight_matrix[index] = np.zeros(100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(df_train.sentiment.values)
y_test = to_categorical(df_test.sentiment.values)
y_val = to_categorical(df_val.sentiment.values)

In [ ]:
import tensorflow
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional

In [ ]:
EMBEDDING_DIM = 100
class_num = 2
model = Sequential()
model.add(Embedding(input_dim = num_words, 
                    output_dim = EMBEDDING_DIM, 
                    input_length = X_train_pad.shape[1],
                    weights = [w2v_weight_matrix], trainable = False))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, return_sequences = True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(200, return_sequences = True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, return_sequences = False)))
model.add(Dense(class_num, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 300, 100)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 300, 200)         160800    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 300, 200)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 300, 400)         641600    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 300, 400)          0

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = ModelCheckpoint('./model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

In [ ]:
Y_train

27156    1
76081    0
26129    1
10074    0
75350    0
        ..
32231    0
96487    0
20095    0
30376    1
50554    1
Name: Sentiment, Length: 79000, dtype: int64

In [ ]:
X_train_pad.shape

(79000, 300)

In [ ]:
Y_val

94532    0
18306    0
22250    0
25757    0
4650     1
        ..
94847    0
22390    1
54426    0
24105    1
94041    0
Name: Sentiment, Length: 1000, dtype: int64

In [ ]:
history_embedding = model.fit(X_train_pad, y_train,
                              epochs = 5, batch_size = 100,
                              validation_data=(X_val_pad, y_val),
                              verbose = 1, callbacks = [es, mc])

Epoch 1/5
790/790 [==============================] - ETA: 0s - loss: 0.4325 - accuracy: 0.8024
Epoch 1: val_accuracy improved from -inf to 0.82500, saving model to ./model.h5
790/790 [==============================] - 130s 164ms/step - loss: 0.4325 - accuracy: 0.8024 - val_loss: 0.3807 - val_accuracy: 0.8250
Epoch 2/5
790/790 [==============================] - ETA: 0s - loss: 0.3592 - accuracy: 0.8412
Epoch 2: val_accuracy improved from 0.82500 to 0.87500, saving model to ./model.h5
790/790 [==============================] - 129s 163ms/step - loss: 0.3592 - accuracy: 0.8412 - val_loss: 0.2995 - val_accuracy: 0.8750
Epoch 3/5
790/790 [==============================] - ETA: 0s - loss: 0.3162 - accuracy: 0.8640
Epoch 3: val_accuracy improved from 0.87500 to 0.88700, saving model to ./model.h5
790/790 [==============================] - 129s 163ms/step - loss: 0.3162 - accuracy: 0.8640 - val_loss: 0.2709 - val_accuracy: 0.8870
Epoch 4/5
790/790 [==============================] - ETA: 0s - l

In [ ]:
model.save('1l_w2v_bilstm_model.h5')

In [ ]:
y_pred = np.argmax(model.predict(X_test_pad), axis = 1)
y_true = np.argmax(y_test, axis = 1)

from sklearn import metrics
print(metrics.classification_report(y_pred, y_true))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89      9919
           1       0.90      0.89      0.89     10081

    accuracy                           0.89     20000
   macro avg       0.89      0.89      0.89     20000
weighted avg       0.89      0.89      0.89     20000



In [ ]:
test1 = ["This is the best pesticide I have ever used, kills pesticide in a manner that you can't imagine",
         "Please don't go for this medicine, Though it produces immediate relief, it causes various side effects that may even kill you"]
testdf = pd.DataFrame({"review":test1})
X_test = tokenizer.texts_to_sequences(testdf.review)
X_test_pad = pad_sequences(X_test, maxlen = 300, padding = 'post')
y_pred = np.argmax(model.predict(X_test_pad), axis = 1)
y_pred

array([1, 0])